# Creating fact table 

## Querying silver data

use storage credential to access data from datalake

In [0]:
df_silver = spark.sql('''select * from parquet.`abfss://silver@carsathvikdatalake.dfs.core.windows.net/carsales`''')

In [0]:
df_model = spark.sql("select * from hive_metastore.gold.dim_model")

df_dealer = spark.sql("select * from hive_metastore.gold.dim_dealer")

df_date = spark.sql("select * from hive_metastore.gold.dim_date")

df_branch = spark.sql("select * from hive_metastore.gold.dim_branch")

In [0]:
df_fact = df_silver.join(df_model, df_silver['Model_ID'] == df_model['Model_ID'],how='left')\
                    .join(df_branch, df_silver['Branch_ID']== df_branch['Branch_ID'],how='left')\
                    .join(df_date, df_silver['Date_ID']==df_date['Date_ID'],how='left')\
                    .join(df_dealer, df_silver['Dealer_ID']==df_dealer['Dealer_ID'],how='left')\
                    .select(df_silver['Revenue'], df_silver['Units_Sold'], df_silver['RevPerUnit'], df_branch['dim_branch_key'], df_date['dim_date_key'], df_dealer['dim_dealer_key'], df_model['dim_model_key'])   

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists('hive_metastore.gold.fact_sales'):
     delta_tbl = DeltaTable.forName(spark, 'hive_metastore.gold.fact_sales')
     delta_tbl.alias('trg').merge(df_fact.alias('src'), 'trg.dim_branch_key = src.dim_branch_key and trg.dim_date_key = src.dim_date_key and trg.dim_dealer_key = src.dim_dealer_key and trg.dim_model_key = src.dim_model_key')\
                           .whenMatchedUpdateAll()\
                            .whenNotMatchedInsertAll()\
                                .execute()

else : 
      df_fact.write.format('delta')\
                   .option('path', 'abfss://gold@carsathvikdatalake.dfs.core.windows.net/fact_sales')\
                   .mode('overwrite')\
                  .saveAsTable('hive_metastore.gold.fact_sales')